In [1]:
from typing import TypedDict, Annotated, Sequence
from typing_extensions import TypedDict

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, FunctionMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain import hub
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# Your further code goes here


In [2]:
import re
import time
import pandas as pd
from pprint import pprint

from gsheet_handler import GSheetHandler
import importlib
module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')

data_raw = pd.read_csv('/home/sftpuser1/uploads/data.csv')
data_raw

no docx
no docx


,link,reference_time,article_time,title,text,author,tags
0,https://corporate.arcelormittal.com/media/news...,2024-05-29 12:53:23.132432,2024-05-22 00:00:00,Prime Minister of Belgium and Federal Minister...,"On May 22, Prime Minister Alexander De Croo an...",NaN,NaN
1,https://corporate.arcelormittal.com/media/news...,2024-05-29 12:53:23.164944,2024-04-02 00:00:00,ArcelorMittal launches ‘The Steel Works’,New content series features insights from Arce...,NaN,"Climate Change,Automotive,Renewables,Infrastru..."
2,https://corporate.arcelormittal.com/media/news...,2024-05-29 12:53:23.164079,2024-04-22 00:00:00,U.S. Department of Energy announces financial ...,"April 22, 2024, Schererville, IN: ArcelorMitta...",NaN,NaN
3,https://corporate.arcelormittal.com/media/news...,2024-05-29 12:53:23.163250,2024-05-10 00:00:00,ArcelorMittal starts the construction of an el...,"La Granda (Gozón), 10 May 2024\nThe constructi...",NaN,NaN
4,https://corporate.arcelormittal.com/media/news...,2024-05-29 12:53:23.162187,2024-05-21 00:00:00,Trial carbon capture unit begins operating on ...,ArcelorMittal and partners Mitsubishi Heavy In...,NaN,NaN
...,...,...,...,...,...,...,...
15894,https://nornickel.ru/news-and-media/press-rele...,2024-05-31 00:14:09.374467,2024-02-08 14:23:00,«Норникель» подписал соглашения о сотрудничест...,«Норникель» подписал соглашения о сотрудничест...,Новости,"Безопасность,Корпоративные новости"
15895,https://nornickel.ru/news-and-media/press-rele...,2024-05-31 00:14:16.056443,2023-12-14 14:30:00,«Норникель» завершил «переезд» «Быстры» в Россию,Новой холдинговой компанией Быстринского ГОКа ...,Новости,Корпоративные новости
15896,https://nornickel.ru/news-and-media/press-rele...,2024-05-31 00:14:21.956760,2023-12-18 19:30:00,При участии «Норникеля» в России будет создано...,"Москва, 18 декабря 2023 г. «Норникель» вошел в...",Пресс-релизы,Корпоративные новости
15897,https://nornickel.ru/news-and-media/press-rele...,2024-05-31 00:14:28.420570,2023-12-21 16:38:00,Дизель-электроход «Норникеля» перевез юбилейну...,Сегодня был зафиксирован новый рекорд грузопер...,Новости,Достижения


In [3]:
df = data_raw.copy()
df['article_time'] = pd.to_datetime(df['article_time'])
filtered_df = df[df['article_time'] == '2024-05-29']
filtered_df

,link,reference_time,article_time,title,text,author,tags
189,https://www.metalbulletin.ru/news/color/10179546/,2024-05-31 01:32:53.484117,2024-05-29,Повышается премия к цене алюминия для японских...,"Как сообщает yieh.com со ссылкой на источники,...",MetalTorg.Ru,NaN
190,https://www.metalbulletin.ru/news/black/10179544/,2024-05-31 01:32:53.486322,2024-05-29,В Мексике сохранили ввозную пошлину на китайск...,Согласно предварительным результатам антидемпи...,MetalTorg.Ru,NaN
191,https://www.metalbulletin.ru/news/scrap/10179535/,2024-05-31 01:32:53.488355,2024-05-29,В Армении продлили запрет на экспорт черного и...,Власти Армении продлили запрет на экспорт черн...,MetalTorg.Ru,NaN
192,https://www.metalbulletin.ru/news/color/10179527/,2024-05-31 01:32:53.490365,2024-05-29,Цветные металлы дорожают на позитивных экономи...,"Во вторник, 28 мая, цены на медь выросли в Лон...",MetalTorg.Ru,NaN
193,https://www.metalbulletin.ru/news/precisiuos/1...,2024-05-31 01:32:53.493016,2024-05-29,Производство серебра KGHM снизилось в апреле н...,"Компания KGHM (Польша) сообщила, что ее продаж...",MetalTorg.Ru,NaN
194,https://www.metalbulletin.ru/news/color/10179523/,2024-05-31 01:32:53.495248,2024-05-29,Eurostat: в марте продолжилось сокращение импо...,"По данным статистической службы Евросоюза, в м...",MetalTorg.Ru,NaN
195,https://www.metalbulletin.ru/news/color/10179521/,2024-05-31 01:32:53.497592,2024-05-29,Китай в январе-апреле почти удвоил импорт алюм...,"По таможенной статистике КНР, импорт необработ...",MetalTorg.Ru,NaN
196,https://www.metalbulletin.ru/news/color/10179522/,2024-05-31 01:32:53.500217,2024-05-29,Vale и бразильский штат Пара на пути к разреше...,Горнопромышленная компания Vale и власти брази...,MetalTorg.Ru,NaN
197,https://www.metalbulletin.ru/news/black/10179513/,2024-05-31 01:32:53.846509,2024-05-29,ТМК выплатит дивиденды по результатам прошедше...,Акционеры ПАО «Трубная металлургическая компан...,MetalTorg.Ru,NaN
198,https://www.metalbulletin.ru/news/ores/10179514/,2024-05-31 01:33:00.408477,2024-05-29,Добыча коксующегося угля в Кузбассе в январе-а...,Индекс австралийского металлургического угля H...,MetalTorg.Ru,NaN


In [4]:
# main.py
import json
import os
import telebot

# Загрузка конфигурации из JSON-файла
with open('config.json') as config_file:
    config = json.load(config_file)

# Установка переменных окружения
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="nlmk_bot"

In [5]:
from datetime import datetime
from langchain import hub

def evaluate_news_article(link, nes_text, news_data, news_topic):
    llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
    prompt = hub.pull("mlenparrot/nlmk_grade")
    today = datetime.now().strftime('%Y-%m-%d')
    params = {
        'link': link,
        'news_text': nes_text,
        'news_data': news_data,
        'news_topic': news_topic,
        'today': today
    }
    chain = prompt | llm
    result = chain.invoke(params)
    return result

result = evaluate_news_article(
    link='https://example.com/news-article',
    nes_text='This is a sample news text.',
    news_data='2024-06-01',
    news_topic='Sample News Topic'
)

print(result)

{'Source_Authority': 4, 'Relevance_for_NLMK': 3, 'Legislative_Changes': 2, 'Novelty_and_Relevance': 4, 'competitor_extraction': [], 'Events_and_Conferences': 3, 'Competitors_and_Startups': 2, 'Products_and_Technologies': 4, 'Impact_on_the_Industry_and_Market': 3}


In [6]:
def apply_evaluation(row):
    result = evaluate_news_article(
        link=row['link'],
        nes_text=row['text'],
        news_data=row['article_time'],
        news_topic=row['title']
    )
    return pd.Series(result)

In [8]:
from tqdm import tqdm
tqdm.pandas()
data_news = filtered_df.copy()
evaluation_results = data_news.progress_apply(apply_evaluation, axis=1)

100%|██████████| 19/19 [00:48<00:00,  2.56s/it]


In [9]:
data_news_rated = pd.concat([data_news, evaluation_results], axis=1).drop(['reference_time', 'author', 'tags'], axis=1)
data_news_rated

,link,article_time,title,text,Source_Authority,Relevance_for_NLMK,Legislative_Changes,Novelty_and_Relevance,competitor_extraction,Events_and_Conferences,Competitors_and_Startups,Products_and_Technologies,Impact_on_the_Industry_and_Market
189,https://www.metalbulletin.ru/news/color/10179546/,2024-05-29,Повышается премия к цене алюминия для японских...,"Как сообщает yieh.com со ссылкой на источники,...",3,1,0,4,[мировой производитель алюминия],0,0,0,3
190,https://www.metalbulletin.ru/news/black/10179544/,2024-05-29,В Мексике сохранили ввозную пошлину на китайск...,Согласно предварительным результатам антидемпи...,4,3,4,5,[],2,0,3,4
191,https://www.metalbulletin.ru/news/scrap/10179535/,2024-05-29,В Армении продлили запрет на экспорт черного и...,Власти Армении продлили запрет на экспорт черн...,3,2,4,4,[],0,0,0,3
192,https://www.metalbulletin.ru/news/color/10179527/,2024-05-29,Цветные металлы дорожают на позитивных экономи...,"Во вторник, 28 мая, цены на медь выросли в Лон...",4,2,0,4,[],3,3,4,4
193,https://www.metalbulletin.ru/news/precisiuos/1...,2024-05-29,Производство серебра KGHM снизилось в апреле н...,"Компания KGHM (Польша) сообщила, что ее продаж...",3,1,0,2,[],0,0,0,2
194,https://www.metalbulletin.ru/news/color/10179523/,2024-05-29,Eurostat: в марте продолжилось сокращение импо...,"По данным статистической службы Евросоюза, в м...",4,3,0,4,[Нидерланды],0,0,0,3
195,https://www.metalbulletin.ru/news/color/10179521/,2024-05-29,Китай в январе-апреле почти удвоил импорт алюм...,"По таможенной статистике КНР, импорт необработ...",4,3,0,4,[Китай],0,0,4,4
196,https://www.metalbulletin.ru/news/color/10179522/,2024-05-29,Vale и бразильский штат Пара на пути к разреше...,Горнопромышленная компания Vale и власти брази...,4,2,0,4,[Vale],0,0,0,3
197,https://www.metalbulletin.ru/news/black/10179513/,2024-05-29,ТМК выплатит дивиденды по результатам прошедше...,Акционеры ПАО «Трубная металлургическая компан...,3,1,0,4,[],0,0,0,2
198,https://www.metalbulletin.ru/news/ores/10179514/,2024-05-29,Добыча коксующегося угля в Кузбассе в январе-а...,Индекс австралийского металлургического угля H...,3,4,0,4,[],0,0,4,3


In [19]:
integer_columns = data_news_rated.select_dtypes(include=['int64']).columns
sum_values = data_news_rated[integer_columns].sum(axis=1)

data_news_rated['rate'] =  sum_values
data_news_rated = data_news_rated.sort_values('rate', ascending=False).reset_index(drop=True) 
data_news_rated

,link,article_time,title,text,Source_Authority,Relevance_for_NLMK,Legislative_Changes,Novelty_and_Relevance,competitor_extraction,Events_and_Conferences,Competitors_and_Startups,Products_and_Technologies,Impact_on_the_Industry_and_Market,rate
0,https://www.metalbulletin.ru/news/black/10179483/,2024-05-29,Tata Steel Netherlands согласовала строительст...,"Как сообщает ценовое агентство Argus, Tata Ste...",4,3,4,5,"[Danieli, Tenova]",3,4,5,4,192
1,https://www.metalbulletin.ru/news/black/10179509/,2024-05-29,Saarstahl нахваливает экологические качества с...,"Как сообщает Recycling Today, железнодорожный ...",4,3,2,5,"[SNCF, Infrabel, Société du Grand Paris]",4,3,5,4,180
2,https://mmk.ru:443/ru/press-center/news/predst...,2024-05-29,Представители ММК выступили на Металлургическо...,Представители Магнитогорского металлургическог...,4,2,0,4,"[ЕВРАЗ, Металлоинвест, Мечел, НЛМК, Северсталь...",5,3,4,3,150
3,https://www.metalbulletin.ru/news/black/10179508/,2024-05-29,Cleveland-Cliffs ведет переговоры о покупке ам...,"Как сообщает агентство Bloomberg, американская...",4,3,0,5,"[United States Steel Corp., Nippon Steel Corp.]",0,4,4,5,150
4,https://www.metalbulletin.ru/news/black/10179544/,2024-05-29,В Мексике сохранили ввозную пошлину на китайск...,Согласно предварительным результатам антидемпи...,4,3,4,5,[],2,0,3,4,150
5,https://www.metalbulletin.ru/news/color/10179527/,2024-05-29,Цветные металлы дорожают на позитивных экономи...,"Во вторник, 28 мая, цены на медь выросли в Лон...",4,2,0,4,[],3,3,4,4,144
6,https://www.metalbulletin.ru/news/black/10179485/,2024-05-29,MEPS : на рынке арматуры в США сохраняется пе...,Как сообщает аналитическое агентство MEPS Inte...,3,2,1,4,[MEPS International Ltd.],2,3,4,3,132
7,https://www.metalbulletin.ru/news/black/10179482/,2024-05-29,"Среди ""стальных лидеров"" только Индия демонстр...","Как сообщает Yieh.com, Индия оказалась единств...",4,3,0,4,"[Китай, Япония, США, Россия]",0,0,5,4,120
8,https://www.metalbulletin.ru/news/color/10179521/,2024-05-29,Китай в январе-апреле почти удвоил импорт алюм...,"По таможенной статистике КНР, импорт необработ...",4,3,0,4,[Китай],0,0,4,4,114
9,https://www.metalbulletin.ru/news/ores/10179514/,2024-05-29,Добыча коксующегося угля в Кузбассе в январе-а...,Индекс австралийского металлургического угля H...,3,4,0,4,[],0,0,4,3,108


In [20]:
ggl.update_gsheet_with_df(data_news_rated,
                          'https://docs.google.com/spreadsheets/d/1cIB0yjbP-uInz7k0rs7zRruyct7C3Cks3P7DUGZTFlc/edit?usp=sharing', 
                          0)

In [29]:
data_news_rated[:3].keys()

Index(['link', 'article_time', 'title', 'text', 'Source_Authority',
       'Relevance_for_NLMK', 'Legislative_Changes', 'Novelty_and_Relevance',
       'competitor_extraction', 'Events_and_Conferences',
       'Competitors_and_Startups', 'Products_and_Technologies',
       'Impact_on_the_Industry_and_Market', 'rate'],
      dtype='object')

In [31]:
data_news_summary = data_news_rated[:3][['link', 'article_time', 'title', 'text']]
data_news_summary

,link,article_time,title,text
0,https://www.metalbulletin.ru/news/black/10179483/,2024-05-29,Tata Steel Netherlands согласовала строительст...,"Как сообщает ценовое агентство Argus, Tata Ste..."
1,https://www.metalbulletin.ru/news/black/10179509/,2024-05-29,Saarstahl нахваливает экологические качества с...,"Как сообщает Recycling Today, железнодорожный ..."
2,https://mmk.ru:443/ru/press-center/news/predst...,2024-05-29,Представители ММК выступили на Металлургическо...,Представители Магнитогорского металлургическог...


In [ ]:
hub.pull("mlenparrot/nlmk_grade")

In [42]:
def summarize_news_article(counts, news_text):
    llm = ChatOpenAI(temperature=0, model="gpt-4o")
    prompt = hub.pull("mlenparrot/nlmk_summary")
    params = {
        'counts': counts,
        'news_text': news_text,
    }
    chain = prompt | llm
    result = chain.invoke(params)
    return result

In [43]:
pd.set_option('display.max_colwidth', None)
news_text = data_news_summary.loc[0, 'text']#.to_string()
pd.reset_option('display.max_colwidth')
news_text

'Как сообщает ценовое агентство Argus, Tata Steel Netherlands подписала соглашения с итальянскими поставщиками сталелитейного оборудования Danieli и Tenova о создании подразделений по производству экологически чистой стали на своем заводе в городе Эймейден.\nКомпания выбрала две итальянские фирмы для оказания инжиниринговых услуг для завода по производству железа прямого восстановления и электродуговой печи (ЭДП) производительностью 3 млн т/год на предприятии Tata Steel в Эймейдене. Tenova будет предоставлять инжиниринговые, снабженческие и консультационные услуги для ЭДП.\nЭтот шаг является частью планов Tata Steel Netherland по достижению углеродной нейтральности к 2045 году. Первый этап плана направлен на сокращение выбросов CO2 на 40% или на 5 мегатонн к 2030 году. Это предполагает замену доменной печи №7 и коксовой печи. -завод с установкой чугуна прямого восстановления и ЭДП.\nTata Steel подаст заявку на получение необходимых разрешений для реализации первого этапа плана до конца

In [44]:
summarize_news_article(500, text_news)

{'summary': 'Tata Steel Netherlands подписала соглашения с итальянскими компаниями Danieli и Tenova для создания экологически чистых сталелитейных подразделений на заводе в Эймейдене. Проект включает строительство завода по производству железа прямого восстановления и электродуговой печи (ЭДП) мощностью 3 млн т/год. Это часть плана по достижению углеродной нейтральности к 2045 году, с сокращением выбросов CO2 на 40% к 2030 году. Tata Steel подаст заявку на разрешения до конца 2024 года.'}

In [50]:
tqdm.pandas()
data_news_summary['summary'] = data_news_summary['text'].progress_apply(lambda x: summarize_news_article(500, x).get('summary', ''))

# Display the updated DataFrame
pd.reset_option('display.max_colwidth')
data_news_summary

100%|██████████| 3/3 [00:13<00:00,  4.34s/it]


,link,article_time,title,text,summary
0,https://www.metalbulletin.ru/news/black/10179483/,2024-05-29,Tata Steel Netherlands согласовала строительст...,"Как сообщает ценовое агентство Argus, Tata Ste...",Tata Steel Netherlands подписала соглашения с ...
1,https://www.metalbulletin.ru/news/black/10179509/,2024-05-29,Saarstahl нахваливает экологические качества с...,"Как сообщает Recycling Today, железнодорожный ...","Saarstahl Group, немецкий производитель стали,..."
2,https://mmk.ru:443/ru/press-center/news/predst...,2024-05-29,Представители ММК выступили на Металлургическо...,Представители Магнитогорского металлургическог...,Представители ММК приняли участие в 21-м Метал...


In [51]:
ggl.update_gsheet_with_df(data_news_summary,
                          'https://docs.google.com/spreadsheets/d/1cIB0yjbP-uInz7k0rs7zRruyct7C3Cks3P7DUGZTFlc/edit?usp=sharing', 
                        1)

In [49]:
data_news_summary.loc[0,'summary'].get('summary', '')

'Tata Steel Netherlands подписала соглашения с итальянскими компаниями Danieli и Tenova для создания экологически чистых сталелитейных подразделений на заводе в Эймейдене. Проект включает строительство завода по производству железа прямого восстановления и электродуговой печи (ЭДП) мощностью 3 млн т/год. Это часть плана по достижению углеродной нейтральности к 2045 году, с сокращением выбросов CO2 на 40% к 2030 году. Tata Steel подаст заявку на разрешения до конца 2024 года.'

In [56]:
pd.set_option('display.max_colwidth', None)
news_text = data_news_summary[['link', 'article_time', 'title', 'summary']].to_string()
pd.reset_option('display.max_colwidth')
news_text

'                                                                                                         link article_time                                                                                           title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        summary\n0                                                           https://www.metalbulletin.ru/news/black/10179483/   2024-05-29  Tata Steel Netherlands согласовала строительство предприятий по производству экологичной стали                  

In [68]:
def tex_generator(news_text):
    llm = ChatOpenAI(temperature=0, model="gpt-4o")
    prompt = hub.pull("mlenparrot/nlmk_tex")
    params = {
        'news_text': news_text,
    }
    chain = prompt | llm
    result = chain.invoke(params)
    return result

In [69]:
tex = tex_generator(news_text)

In [70]:
tex_code = tex.get('tex_code', '')

In [71]:
tex_code

'\\documentclass[a4paper,12pt]{article}\n\\usepackage{fontspec}\n\\usepackage[utf8]{inputenc}\n\\usepackage{hyperref}\n\\usepackage{geometry}\n\\usepackage{titlesec}\n\\hypersetup{\n    colorlinks=false,\n    pdfborder={0 0 0},\n}\n\\geometry{margin=1in}\n% Title formatting\n\\titleformat{\\section}\n  {\\normalfont\\fontsize{16}{18}\\bfseries}{\\thesection}{1em}{}\n\\titleformat{\\subsection}\n  {\\normalfont\\fontsize{14}{16}\\bfseries}{\\thesubsection}{1em}{}\n\\titleformat{\\subsubsection}\n  {\\normalfont\\fontsize{12}{14}\\bfseries\\itshape}{\\thesubsubsection}{1em}{}\n\\title{\\LARGE \\textbf{Управление новостной рассылкой Генеративные ИИ случаи}}\n\\date{\\small 5 июня 2024}\n\\author{}\n\\begin{document}\n\\maketitle\n\\section*{\\large Тренды Генеративного ИИ}\n\\subsection*{\\normalsize \\href{https://www.metalbulletin.ru/news/black/10179483/}{Tata Steel Netherlands согласовала строительство предприятий по производству экологичной стали}}\n\\textit{\\small Источник: Metal Bu

In [72]:
import pandas as pd
from jinja2 import Template
import subprocess
import os


tex_filename = "newsletter.tex"
with open(tex_filename, "w") as f:
    f.write(tex_code)

# Компиляция LaTeX в PDF
subprocess.run(["pdflatex", tex_filename])

# Удаление временных файлов, созданных pdflatex
for ext in [".aux", ".log", ".out"]:
    os.remove(f"newsletter{ext}")

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./newsletter.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size12.clo))
(/usr/share/texlive/texmf-dist/tex/latex/fontspec/fontspec.sty
(/usr/share/texlive/texmf-dist/tex/latex/l3packages/xparse/xparse.sty
(/usr/share/texlive/texmf-dist/tex/latex/l3kernel/expl3.sty
(/usr/share/texlive/texmf-dist/tex/latex/l3backend/l3backend-pdftex.def)))

! Fatal Package fontspec Error: The fontspec package requires either XeTeX or
(fontspec)                      LuaTeX.
(fontspec)                      
(fontspec)                      You must change your typesetting engine to,
(fontspec)                      e.g., "xelatex" or "lualatex" instead of


FileNotFoundError: [Errno 2] No such file or directory: 'newsletter.aux'